In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))

from Utils.Solver import *
from Utils.Signals import *

In [ ]:
# Let's assume we are interested in the following stocks: AAPL, MSFT, TSLA, AMZN, GOOG
tickers = ['AAPL', 'MSFT', 'TSLA', 'AMZN', 'GOOG', 'IBM', 'NFLX', 'NVDA', 'AMD', 'INTC', 'TXN', 'QCOM', 'MU', 'ADI', 'HPQ', 'GLW', 'AVGO', 'ADP', 'INTU', 'PFE'] 

# Download historical stock data
data = yf.download(tickers, start='2010-01-01', end='2025-01-01')
# Initialize the portfolio solver with appropriate penalty and max weight threshold
portfolio_solver = Portfolio_Solver(0.8, max_weight_threshold=0.2 )

In [ ]:
start_date_signal = '2000-01-01'
end_date_signal = '2021-01-01'
date_range_signal = pd.date_range(start=start_date_signal, end=end_date_signal)

start_date_eval = '2019-01-01'
end_date_eval = '2020-01-01'
date_range_eval = pd.date_range(start=start_date_eval, end=end_date_eval)

display(date_range_eval)

In [ ]:
# Initialize an empty list to store the rows for the first DataFrame
dataset_scores = []

# Step 1: Create the DataFrame with rsi_scores, macd_scores, and sma_scores
for date in date_range_signal:

    # Step 1a: Calculate the RSI signal scores
    rsi_signal_scores = calculate_rsi_signal(data, tickers, date=date, period=14)
    rsi_scores = np.array([score[1] for score in rsi_signal_scores])

    # Step 1b: Calculate the MACD signal scores
    macd_signal_scores = calculate_macd_signal(data, tickers, date=date)
    macd_scores = np.array([score[1] for score in macd_signal_scores])

    # Step 1c: Calculate the SMA signal scores
    sma_signal_scores = calculate_sma_signal(data, tickers, date=date)
    sma_scores = np.array([score[1] for score in sma_signal_scores])

    print(f"processing {date} : {rsi_scores}, {macd_scores}, {sma_scores} ")

    if np.any(np.isnan(rsi_scores)) or np.any(np.isnan(macd_scores)) or np.any(np.isnan(sma_scores)):
        #print(f"Skipping {date} due to NaN values in the signals.")
        continue  # Skip this date and move to the next one

    

    # Step 1d: Add the scores to the first dataset
    dataset_scores.append({
        'date': date,
        'rsi_scores': rsi_scores,
        'macd_scores': macd_scores,
        'sma_scores': sma_scores
    })


# Convert the first dataset into a DataFrame
df_scores = pd.DataFrame(dataset_scores)
display(df_scores.head)

processing 2000-09-04 00:00:00 : [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan], [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan], [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan] 
processing 2000-09-05 00:00:00 : [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan], [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan], [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan] 
processing 2000-09-06 00:00:00 : [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan], [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan], [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan] 
processing 2000-09-07 00:00:00 : [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan], [nan nan nan nan nan nan nan

In [ ]:
start_date_train = '2018-01-01'
end_date_train = '2019-01-01'
date_range_train = pd.date_range(start=start_date_train, end=end_date_train)

start_date_eval = '2020-01-01'
end_date_eval = '2021-01-01'
date_range_eval = pd.date_range(start=start_date_eval, end=end_date_eval)

# Filter the dataframe within the date range
df_train = df_scores[(df_scores['date'] >= start_date_train) & (df_scores['date'] <= end_date_train)]
display(df_train)
df_eval = df_scores[(df_scores['date'] >= start_date_eval) & (df_scores['date'] <= end_date_eval)]
display(df_eval)

In [ ]:
# Initialize weights with small random values or equal weights
signal_weights = np.array([1/3, 1/3, 1/3])  # [w_rsi, w_macd, w_sma]


avg_annualized_return, total_return_sum = portfolio_solver. calculate_eval_returns(
    tickers, data, df_eval, signal_weights
)


In [ ]:
print(avg_annualized_return, total_return_sum)